In [6]:
import os
import cv2
import numpy as np
import networkx as nx
import torch
import torchvision.transforms as transforms
from skimage.segmentation import felzenszwalb
from skimage.feature.texture import graycomatrix, graycoprops
from skimage.color import rgb2gray
from tqdm import tqdm

In [5]:
import os
import cv2 as cv
import numpy as np
import pandas as pd
import skimage.feature as skf

In [7]:
def GraphSegmentation(image):
    image = cv.cvtColor(image, cv.COLOR_BGR2RGB)  # Convert to RGB for display

    # Define mask for GrabCut
    row, col, _ = image.shape
    mask = np.zeros((row, col), np.uint8)

    # Initialize models correctly
    bgdModel = np.zeros((1, 65), np.float64)  # Background model
    fgdModel = np.zeros((1, 65), np.float64)  # Foreground model

    x0, y0 = 1, 1  # Ensure valid rectangle coordinates
    x1, y1 = col , row
    rect = (x0, y0, x1 - x0, y1 - y0)

    # Run GrabCut
    cv.grabCut(image, mask, rect, bgdModel, fgdModel, 10, cv.GC_INIT_WITH_RECT)

    # Modify mask to extract foreground
    mask2 = np.where((mask == 2) | (mask == 0), 0, 1).astype("uint8")
    segmented = image * mask2[:, :, np.newaxis]
    return segmented
        


In [12]:
def ExtractFeatures(image):

    if len(image.shape) == 3:  
        image = cv.cvtColor(image, cv.COLOR_BGR2GRAY)

    edges = cv.Canny(image, 50, 150)
    edge_count = np.sum(edges > 0)

    _, binary_mask = cv.threshold(image, 0, 255, cv.THRESH_BINARY + cv.THRESH_OTSU)
    binary_mask = binary_mask // 255  # Normalize to 0 and 1
    
    crack_pixels = np.sum(binary_mask)
    total_pixels = binary_mask.size
    crack_ratio = crack_pixels / total_pixels

    glcm = skf.graycomatrix(image, distances=[1], angles=[0], levels=256, symmetric=True, normed=True)
    contrast = skf.graycoprops(glcm, 'contrast')[0, 0]
    correlation = skf.graycoprops(glcm, 'correlation')[0, 0]
    energy = skf.graycoprops(glcm, 'energy')[0, 0]
    homogeneity = skf.graycoprops(glcm, 'homogeneity')[0, 0]
    return [crack_ratio, edge_count, contrast, correlation, energy, homogeneity]




In [30]:
import os
import cv2
import numpy as np
import pandas as pd
from tqdm import tqdm  # Import tqdm for progress bar

# Define the dataset path and categories
dataset_path = r"C:\Users\EDWIN\OneDrive\Desktop\Study materials\SEM-6\MV Project\Dataset"
categories = ["Positive", "Negative"]
max_images_per_category = 5000


def process_images():
    data = []
    labels = []

    for category in categories:
        label = 1 if category.lower() == "positive" else 0
        folder_path = os.path.join(dataset_path, category)
        img_list = os.listdir(folder_path)

        # Use tqdm to show progress
        for img_name in tqdm(img_list[:max_images_per_category], desc=f"Processing {category} images"):
            img_path = os.path.join(folder_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

            if img is None:
                continue

            segmented = GraphSegmentation(img)
            features = ExtractFeatures(segmented)

            data.append(features)
            labels.append(label)

    data = np.array(data)
    labels = np.array(labels)

    return data, labels

X, y = process_images()



Processing Positive images:   0%|          | 0/5000 [00:00<?, ?it/s]

Processing Negative images: 100%|██████████| 5000/5000 [1:03:11<00:00,  1.32it/s]


In [33]:

feature_names = ['Crack Ratio', 'Edge Count', 'Contrast', 'Correlation', 'Energy', 'Homogeneity']
df = pd.DataFrame(X, columns=feature_names)
df['Label'] = y


df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Save to Excel
output_path = r"C:\Users\EDWIN\OneDrive\Desktop\Study materials\SEM-6\MV Project\processed_data.xlsx"
df.to_excel(output_path, index=False)

print(f"Shuffled data saved to: {output_path}")

Shuffled data saved to: C:\Users\EDWIN\OneDrive\Desktop\Study materials\SEM-6\MV Project\processed_data.xlsx


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Load the dataset
file_path = r"C:\Users\EDWIN\OneDrive\Desktop\Study materials\SEM-6\MV Project\processed_data.xlsx"
df = pd.read_excel(file_path)

# Separate features and labels
X = df.drop(columns=["Label"]).values  # Features
y = df["Label"].values  # Labels

ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.